In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import pandas as pd

# Import graph
import pickle

# Graph Embeddings (using node2vec)
#from node2vec import Node2Vec

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/whole_final_dataset.csv')
data.head()

,idx,node,isp,in_degree,out_degree,pagerank,weightsout,weightsin,numout,numin
0,0,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0,215.0,1624.0,6.168217e-07,14285.789338,15931.196110,215.0,1624.0
1,1,0x1266f8b9e4dffc9e2f719bf51713f7e714516861,0,1.0,1.0,1.333369e-07,2.344203,2.344623,1.0,1.0
2,2,0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb,1,8.0,0.0,8.449769e-07,0.000000,25.268098,8.0,0.0
3,3,0x256fc19e9d8f5be0d451841f218289d1adbbaaa3,0,0.0,1.0,1.331609e-07,0.489485,0.000000,0.0,1.0
4,4,0xb50d0c4cb2c29cc232c96a59e9c65eb82914ec75,0,63.0,166.0,8.827056e-07,19.336007,8.408347,63.0,166.0


In [ ]:
def load_pickle(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

G = load_pickle('/content/drive/My Drive/BT4012 Team 8/Ethereum/subgraph.pkl')
G_simple = nx.Graph(G)

# Clustering coefficient

In [ ]:
clustering_coefficient = nx.clustering(G_simple)

# Convert the dictionary to a Pandas DataFrame
clustering_coefficient_df = pd.DataFrame.from_dict(clustering_coefficient, orient='index', columns=['Clustering Coefficient']).reset_index()
clustering_coefficient_df.columns=['node','Clustering Coefficient']

In [ ]:
clustering_coefficient_df.to_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/clustering_coefficient.csv',index=False)

## Centrality Features

In [ ]:
# Step 1: Centrality Features

# Calculate centrality measures for each node
closeness_centrality = nx.closeness_centrality(G)
# If you're sure your data can be represented as a simple graph

# Now you can calculate betweenness centrality for the simple graph
betweenness_centrality = nx.betweenness_centrality(G_simple)
eigenvector_centrality = nx.eigenvector_centrality(G_simple, max_iter=1000)

## Components

In [ ]:
import json
def output(component_characteristics, node_component_characteristics):

# ... (the previous code for calculating characteristics)

# Output component characteristics to a JSON file
  with open('/content/drive/My Drive/BT4012 Team 8/Ethereum/component_characteristics.json', 'w') as component_file:
      json.dump(component_characteristics, component_file, indent=4)

# Output node-specific component characteristics to a JSON file
  with open('/content/drive/My Drive/BT4012 Team 8/Ethereum/node_component_characteristics.json', 'w') as node_component_file:
      json.dump(node_component_characteristics, node_component_file, indent=4)

In [ ]:
# Find the connected components
connected_components = list(nx.connected_components(G_simple))

# Calculate the number of connected components
num_connected_components = len(connected_components)

print(f"Number of Connected Components: {num_connected_components}")

In [ ]:
# Assuming you have your simplified graph G_simple

# Calculate network characteristics
num_connected_components = nx.number_connected_components(G_simple)
network_density = nx.density(G_simple)


# Create dictionaries to store component characteristics
component_characteristics = {}
node_component_characteristics = {}

connected_components = list(nx.connected_components(G_simple))

# Enumerate starting from the third component (index 2)
for i, component in enumerate(connected_components):
    component_subgraph = G_simple.subgraph(component)

    # Calculate component characteristics
    component_characteristics[i] = {
        'component_size': len(component),
        'component_diameter': nx.diameter(component_subgraph),
        'component_eccentricity': nx.eccentricity(component_subgraph),
        'component_avg_degree': sum(dict(component_subgraph.degree()).values()) / len(component),
        'component_clustering_coefficient': nx.average_clustering(component_subgraph)
    }

    output(component_characteristics, node_component_characteristics)

    # Associate component ID with nodes
    for node in component:
        node_component_characteristics[node] = {
            'component_id': i
        }
    output(component_characteristics, node_component_characteristics)

## Network characteristics

In [ ]:
print(num_connected_components)
print(np.mean(avg_path_length_list))
print(network_density)

## Graph Embeddings

In [ ]:
# Step 10: Graph Embeddings (using node2vec)
from node2vec import Node2Vec
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
# Now you can get embeddings for nodes using model.wv

In [ ]:
# Step 12: Combine Features
# Create a feature matrix (X) by combining all the features you've engineered
# You need to create a list of dictionaries where each dictionary represents the features for a node.
# For example:
features = []
for node in G.nodes():
    feature_dict = {
        'node':node,
        'closeness_centrality': closeness_centrality[node],
        'betweenness_centrality': betweenness_centrality[node],
        'eigenvector_centrality': eigenvector_centrality[node],
        'num_connected_components': num_connected_components,
        'average_path_length': np.mean(avg_path_length_list),
        'network_density': network_density
        # Add more features as needed
    }
    features.append(feature_dict)

# Now you have a list of feature dictionaries that you can convert into a Pandas DataFrame or use with your machine learning model.


df = pd.DataFrame(features)

# Now you can use df as your feature matrix for training your model.

In [ ]:
df.to_csv("Centrality_network_features.csv", index=False)

# Temporal features

In [4]:
temporal_features = pd.read_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/edge_new2.csv')
temporal_features = temporal_features[['fromnode', 'tonode', 'timestamp']].drop_duplicates()
temporal_features['timestamp'] = pd.to_datetime(temporal_features['timestamp'])
temporal_features.sort_values(['fromnode', 'tonode', 'timestamp'], inplace=True)
#temporal_features['time_since_last_interaction'] = temporal_features.groupby(['fromnode', 'tonode'])['timestamp'].diff()

In [6]:
# Calculate the total number of interactions for each node
interaction_count = temporal_features.groupby('fromnode')['tonode'].count().reset_index()
interaction_count.columns = ['fromnode', 'interaction_count']

In [ ]:
# Count interactions within different time windows
day_interactions = temporal_features[temporal_features['timestamp'] > (temporal_features['timestamp'].max() - pd.DateOffset(days=1))]
week_interactions = temporal_features[temporal_features['timestamp'] > (temporal_features['timestamp'].max() - pd.DateOffset(weeks=1))]

# Count interactions within the last day and last week
day_interactions_count = day_interactions.groupby('fromnode')['tonode'].count().reset_index()
week_interactions_count = week_interactions.groupby('fromnode')['tonode'].count().reset_index()

day_interactions_count.columns = ['fromnode', 'interactions_last_day']
week_interactions_count.columns = ['fromnode', 'interactions_last_week']


In [ ]:
interaction_count = pd.merge(interaction_count, day_interactions_count, on='fromnode',how='outer')
interaction_count = pd.merge(interaction_count, week_interactions_count, on='fromnode',how='outer')
interaction_count.to_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/interaction_count.csv',index=False)

In [ ]:
# Calculate the number of unique days with interactions for each node
active_days = temporal_features.groupby('fromnode')['timestamp'].apply(lambda x: x.dt.date.nunique()).reset_index()
active_days.columns = ['fromnode', 'active_days']
active_days.to_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/active_days.csv',index=False)

In [ ]:
# Define business hours (e.g., 9 AM to 5 PM)
business_hours_interactions = temporal_features[
    (temporal_features['timestamp'].dt.hour >= 9) &
    (temporal_features['timestamp'].dt.hour <= 17)
]
business_hours_interactions_count = business_hours_interactions.groupby('fromnode')['tonode'].count().reset_index()
business_hours_interactions_count.columns = ['fromnode', 'business_hours_interactions_count']
business_hours_interactions_count.to_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/business_hours_interactions_count.csv',index=False)

In [ ]:
# Interactions per hour
temporal_features['hour'] = temporal_features['timestamp'].dt.hour
interactions_per_hour = temporal_features.groupby(['fromnode', 'hour'])['tonode'].count().unstack().fillna(0)
interactions_per_hour.reset_index().to_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/interactions_per_hour.csv',index=False)

In [ ]:
interaction_count = pd.read_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/interaction_count.csv')
active_days= pd.read_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/active_days.csv')
business_hours_interactions_count= pd.read_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/business_hours_interactions_count.csv')
interactions_per_hour = pd.read_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/interactions_per_hour.csv')

In [ ]:
temporal_features = pd.merge(temporal_features[['fromnode']].drop_duplicates(), interaction_count, on='fromnode',how='outer')
temporal_features = pd.merge(temporal_features, active_days, on='fromnode',how='outer')
temporal_features = pd.merge(temporal_features, business_hours_interactions_count, on='fromnode',how='outer')
temporal_features = pd.merge(temporal_features, interactions_per_hour, on='fromnode',how='outer')

In [ ]:
temporal_features.business_hours_interactions_count = temporal_features.business_hours_interactions_count.fillna(0)
temporal_features.drop(['interactions_last_day',
       'interactions_last_week'], axis=1, inplace=True)

In [ ]:
temporal_features.drop_duplicates().to_csv('/content/drive/My Drive/BT4012 Team 8/Ethereum/additional_temporal_features.csv',index=False)